In [2]:
#link google colab with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
#first to import libraries
from keras.layers import Input, Lambda, Dense, Flatten, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix


import warnings   
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
train_data_dir = '/content/drive/MyDrive/Data_Small2/SRGAN_images/train'
val_data_dir = '/content/drive/MyDrive/Data_Small2/SRGAN_images/val'
test_data_dir = '/content/drive/MyDrive/Data_Small2/SRGAN_images/test'
img_width, img_height = 128, 128
batch_size = 32

In [5]:
# Preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [23]:
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

Found 2014 images belonging to 2 classes.
Found 302 images belonging to 2 classes.
Found 514 images belonging to 2 classes.


In [24]:
# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [25]:
# Add new layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [26]:
# Create a new model
modelB = Model(inputs=base_model.input, outputs=predictions)

In [27]:
# Freeze layers
for layer in base_model.layers:
    layer.trainable = False 

In [28]:
# Compile the model
modelB.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [29]:
# Train the model
modelB.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=150,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size)

<ipython-input-29-8a0d9c09a9b6>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  modelB.fit_generator(


Epoch 1/150
62/62 [==============================] - 5s 68ms/step - loss: 0.6790 - accuracy: 0.5732 - val_loss: 0.5271 - val_accuracy: 1.0000
Epoch 2/150
62/62 [==============================] - 4s 66ms/step - loss: 0.6663 - accuracy: 0.6014 - val_loss: 0.5172 - val_accuracy: 1.0000
Epoch 3/150
62/62 [==============================] - 4s 66ms/step - loss: 0.6616 - accuracy: 0.6019 - val_loss: 0.4913 - val_accuracy: 1.0000
Epoch 4/150
62/62 [==============================] - 4s 62ms/step - loss: 0.6583 - accuracy: 0.6019 - val_loss: 0.5188 - val_accuracy: 1.0000
Epoch 5/150
62/62 [==============================] - 4s 62ms/step - loss: 0.6566 - accuracy: 0.6014 - val_loss: 0.4763 - val_accuracy: 1.0000
Epoch 6/150
62/62 [==============================] - 5s 75ms/step - loss: 0.6549 - accuracy: 0.6009 - val_loss: 0.4777 - val_accuracy: 1.0000
Epoch 7/150
62/62 [==============================] - 4s 63ms/step - loss: 0.6525 - accuracy: 0.6039 - val_loss: 0.4785 - val_accuracy: 1.0000
Epoch 

In [30]:
modelB.save('my_modelB.h5')

In [40]:
modelB.save('/content/drive/MyDrive/models/modelB')
